In [1]:
from sktime.transformations.series.difference import Differencer
from sktime.forecasting.compose import TransformedTargetForecaster, make_reduction
from xgboost import XGBRegressor

def build_forecaster(window_length=10, random_state=42):
    reg = XGBRegressor(
        n_estimators=500,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        objective="reg:squarederror",
        random_state=random_state,
        n_jobs=1,
    )
    forecaster = TransformedTargetForecaster(steps=[
        ("diff", Differencer(lags=1)),
        ("regr", make_reduction(
            estimator=reg,
            strategy="recursive",
            window_length=window_length
        )),
    ])
    return forecaster

base_forecaster = build_forecaster()

In [2]:
import pandas as pd
from datetime import datetime, timezone
import numpy as np

d = pd.DataFrame()
d['timestamp'] = pd.date_range(start=datetime(year=2025, month=1, day=1, tzinfo=timezone.utc), end=datetime.now(tz = timezone.utc), freq='5min')
d['data'] = np.random.randn(len(d))
d.set_index('timestamp', inplace=True)

fit = d.loc[d.index[:500]]
predict = d.loc[d.index[501:1000]]
update = d.loc[d.index[1001:1500]]

predict['data'] = np.nan

base_forecaster.fit(y=fit)
base_forecaster.predict(fh=len(predict))
base_forecaster.update(y=predict, update_params=False)
base_forecaster.update(y=update)

ValueError: Inferred frequency None from passed values does not conform to passed frequency 5min